#### 멀티미디어 저장을 위해 json 파일 전체를 받아오는 방법을 사용하기로 함
- 참고 : https://stackabuse.com/accessing-the-twitter-api-with-python/

In [1]:
from twython import Twython              # pip install twython
import json
import pandas as pd

In [2]:
credentials = {}
credentials['CONSUMER_KEY'] = "-"
credentials['CONSUMER_SECRET'] = "-"  
credentials['ACCESS_TOKEN'] = "-"
credentials['ACCESS_SECRET'] = "-"

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [3]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query : 각자 원하는 옵션값 사용하면 됨.
query = {'screen_name': 'moonriver365',
        'count': '10',
         'tweet_mode' : 'extended',    # 이거 안하면 full_text가 안나옴
         'includ_rts' : 'False',
         'exclude_replies' : 'True',
         'trim_user' : 'True'
        }

# 추후에 query에 max_id를 추가하고자 할 때  : query.update({'max_id' : '원하는 값'})

#### getUserTimeline()때와는 달리 모든 정보가 담겨있는 것을 확인할 수 있다.

In [4]:
python_tweets.get_user_timeline(**query)[0]

{'created_at': 'Mon Oct 12 08:48:58 +0000 2020',
 'id': 1315575174964469760,
 'id_str': '1315575174964469760',
 'full_text': '이번의 방역완화 조치는 \n우리가 코로나를 방역의 통제 속에 \n둘 수 있다는 자신감에 근거한 것입니다.\n\n전 세계적인 코로나 위기 속에서 \n우리는 방역과 경제 모두에서 \n강한 면모를 보여주며 \n‘위기에 강한 나라, 대한민국’으로 \n국가적 위상이 높아지고 있습니다. \n모두가 국민 덕분입니다. https://t.co/HNwRIjijcv',
 'truncated': False,
 'display_text_range': [0, 161],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1315574506572767232,
    'id_str': '1315574506572767232',
    'indices': [162, 185],
    'media_url': 'http://pbs.twimg.com/media/EkHcCyPUYAAtMt6.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/EkHcCyPUYAAtMt6.jpg',
    'url': 'https://t.co/HNwRIjijcv',
    'display_url': 'pic.twitter.com/HNwRIjijcv',
    'expanded_url': 'https://twitter.com/moonriver365/status/1315575174964469760/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'small': {'w': 680, 'h': 45

In [9]:
# 멀티미디어 정보를 추출하고 싶다면
python_tweets.get_user_timeline(**query)[0]['extended_entities']

{'media': [{'id': 1315574506572767232,
   'id_str': '1315574506572767232',
   'indices': [162, 185],
   'media_url': 'http://pbs.twimg.com/media/EkHcCyPUYAAtMt6.jpg',
   'media_url_https': 'https://pbs.twimg.com/media/EkHcCyPUYAAtMt6.jpg',
   'url': 'https://t.co/HNwRIjijcv',
   'display_url': 'pic.twitter.com/HNwRIjijcv',
   'expanded_url': 'https://twitter.com/moonriver365/status/1315575174964469760/photo/1',
   'type': 'photo',
   'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
    'small': {'w': 680, 'h': 453, 'resize': 'fit'},
    'medium': {'w': 1200, 'h': 800, 'resize': 'fit'},
    'large': {'w': 1575, 'h': 1050, 'resize': 'fit'}}}]}

#### 우선 내가 사용한 방법
1. 모든 json파일을 List에 저장하고
2. 그 List에서 내가 필요한 것을 media type에 따라 추출하는 코드 작성

In [ ]:
def get_all_tweets():
    global tweetLists
    for status in python_tweets.get_user_timeline(**query):       
        tweetLists.append(status)
        print(status['id'])
        while len(status) > 0 :
            status = python_tweets.get_user_timeline(**query)[0]
            oldest = status['id'] - 1
            
            # 다음 for문을 위해서 max_id 업데이트 : 매번 새 값으로 덮어씌워짐
            query.update({'max_id' : oldest})
            
            tweetLists.append(status)
            print(oldest)
    return tweetLists

#### 더 깔끔한 코드 작성을 해야겠지만 우선 남겨둔다.

In [ ]:
idLists = []
dateLists = []
textLists = []
hashtagLists = []
urlLists = []

for tweet in tweetLists :
    try :
        # media type이 photo와 video가 있는데 상황에 따라 추출해야하는 url이 다르기 때문에 조건문 작성
        if tweet['extended_entities']['media'][0]['type'] == 'photo' : 
            i = 0
            # 사진의 경우 최대 4장까지 포함될 수 있으므로 모든 이미지 주소를 수집할 수 있도록
            len_media = len(tweet['extended_entities']['media'])
            while i < len_media:
                url = tweet['extended_entities']['media'][i]['media_url']
                i += 1
                dateLists.append(tweet['created_at'])
                idLists.append(tweet['id'])
                textLists.append(tweet['full_text'])
                urlLists.append(url)
                
                # 그리고 hashtag 정보가 있으면 이것 또한 수집하기로 함
                if tweet['entities']['hashtags'] :
                    hashtagLists.append(tweet['entities']['hashtags'][0]['text'])
                else :
                    hashtagLists.append("")
        else :
            url = tweet['extended_entities']['media'][0]['video_info']['variants'][0]['url']
            dateLists.append(tweet['created_at'])
            idLists.append(tweet['id'])
            textLists.append(tweet['full_text'])
            urlLists.append(url)
            
            # 그리고 hashtag 정보가 있으면 이것 또한 수집하기로 함
            if tweet['entities']['hashtags'] :
                hashtagLists.append(tweet['entities']['hashtags'][0]['text'])
            else :
                hashtagLists.append("")
    
    # 멀티미디어가 없는 트위터는 if 조건에 걸리는 tweet['extended_entities']가 없어서 오류가 발생하므로
    # 예외문 작성해서 마찬가지로 List에 추가되도록 만듦
    except KeyError : 
        dateLists.append(tweet['created_at'])
        idLists.append(tweet['id'])
        textLists.append(tweet['full_text'])
        urlLists.append("")
        
         # 그리고 hashtag 정보가 있으면 이것 또한 수집하기로 함
        if tweet['entities']['hashtags'] :
            hashtagLists.append(tweet['entities']['hashtags'][0]['text'])
        else :
            hashtagLists.append("")

##### 시간에 관한 함수 참고 : https://docs.python.org/ko/3/library/datetime.html

In [12]:
import datetime

In [32]:
# 날짜 포맷을 바꾸려고 함 : ''Mon Oct 12 08:48:58 +0000 2020'→ datetime.datetime(2020, 10, 12, 8, 48, 58)
test_time = python_tweets.get_user_timeline(**query)[0]['created_at']
test_time2 = datetime.datetime.strptime(test_time, "%a %b %d %I:%M:%S +0000 %Y")

# 왜 이렇게 바꾸냐면 이미지 저장할 때 파일명으로 시간을 사용하고자 함
test_time2.strftime("%Y-%m-%d-%H-%M")

'2020-10-12-08-48'

In [ ]:
# 데이터 프레임을 만들었는데 너무 오랜만이라 방법이 생각이 안난다. 추후에 더 좋은 방법으로 수정할 것.
df = pd.DataFrame([idLists, dateLists, textLists, hashtagLists, urlLists]).transpose()
df.columns =  ['id', 'date', 'text', 'hashtag', 'media_url'];df.tail()

In [ ]:
# 자잘한 정제 과정은 생략
df.to_csv('파일명.csv', index = False, encoding = 'utf-8-sig') # text가 이상해서 encoding 추가

#### 추출한 media_url을 이용해서 멀티미디어 저장하기
- 참고 : https://towardsdatascience.com/automatic-collection-of-images-under-rickyrenuncia-in-twitter-a86adb7124cb

In [ ]:
import os
import urllib.request

In [ ]:
folder_name = "img_backup"
errorLists = []

for tweet in df.itertuples() :
    try : 
        tag = tweet[4]
        iid = tweet[1]
        time = tweet[2]
        
        # 만약 media_url이 있다면 저장할 것
        if tweet[5] : 
            file_name = str(time)  + "_" + str(iid) + "_" + tag
            url = tweet[5]
            
            # .jpg와 .mp4에 따라서 
            file_type = os.path.splitext(url)[::-1][0].split("?")[0]
            urllib.request.urlretrieve(url, os.path.join(folder_name, file_name + file_type))
    except :
        errorLists.append([iid, tag, time, tweet[5]])
        pass